In [1]:
%load_ext autoreload
%autoreload 2

from preprocessing import oversample_minority_classes
import numpy as np; np.random.seed(0)
from config import OUTPUT_DIR
from util import *
from AL import AL_split, mAP_and_APs, evaluation_dict, AL_resnet16_simulation
import pickle

from config import * 
from util import *
from preprocessing import oversample_minority_classes, build_resnet16
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve
from scikeras.wrappers import KerasClassifier
from modAL import ActiveLearner
import pickle
from tensorflow import keras
import keras_cv
from keras import metrics
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score

from util import MULTICLASS_LABELS
from sklearn.metrics import average_precision_score



2024-10-03 09:27:15.484195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 09:27:15.484229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 09:27:15.485312: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 09:27:15.495221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 09:27:16.759131: W tensorflow/comp

In [ ]:
X = np.load(INTERMEDIATE / 'logmel_multiclass_noise.npy')
Y = np.load(INTERMEDIATE / 'logmel_labels_multiclass_noise.npy')

In [ ]:

working_dir = '..'
query_method = ...
budget_cap = ...
query_size = ...

working_dir.mkdir(exist_ok=True)
keras.utils.set_random_seed(0) # reproducability

# split data
init, pool, test = AL_split(X, Y)
initial_X, initial_Y = init
pool_X, pool_Y = pool
test_X, test_Y = test
currently_labelled = len(initial_X)
initial_ds_size = currently_labelled + len(pool_X)

query_size = int(((pool_X.shape[0] + initial_X.shape[0]) * budget_cap) / n_queries)
print(f'query size: {query_size}')
LB_metrics = []

# !!! oversample inital only 
initial_X, initial_Y = oversample_minority_classes(initial_X, initial_Y)
import gc; gc.collect()

labeled_X, labeled_Y = initial_X, initial_Y

for query in range(1, 5+1):
    
    # new model at each epoch
    model = build_resnet16((40, 107, 1))
    model.compile(optimizer='adam',
            loss=keras_cv.losses.FocalLoss(alpha=0.25, gamma=2), 
            metrics=[
                metrics.Recall(thresholds=0.5), # NOTE: useless: macro
                metrics.Precision(thresholds=0.5), # NOTE: useless: macro
                metrics.AUC(curve='pr', name='auc_pr') # NOTE: useless: macro
            ])


    
    query_indices, _ =   query_method()
    
    pool_X, pool_Y = oversample_minority_classes(pool_X, pool_Y)
    
